実装にあたってこちらの記事を参考にさせていただきました  
[Zenn 歩行時の加速度,角速度データから歩行軌跡を描いてみた - Beyond Toyama](https://zenn.dev/happy663/articles/1a0abc285b3a33)

# 事前準備

## ライブラリのインポート

In [131]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [143]:
import time
import requests
import random
import csv

# API実行時間計測

In [138]:
url = "http://localhost:8082/api/v1/stayers"
baseURL = "http://localhost:8082"

# ヘッダー
headers = {
    "Content-Type": "application/json"
}

## ユーザ登録用関数の定義（APIを用いて）

In [149]:
def createBasicBeaconUsers(numUsers, startUserId):
  for i in range(numUsers):
    name = ''.join(random.choices('abcdefghijklmnopqrstu', k=7))
    # uuid = ''.join(random.choices('0123456789abcdef', k=5))
    uuid = "{:05}".format(startUserId + i)
    data = {
        "name":name,
        "uuid":uuid,
        "email":"",
        "role":1,
        "communityId":1,
        "beaconName":"FCS1301",
        "tagIds":[1,5,10]
    }
    response = requests.post(baseURL+"/api/v1/users", json=data, headers=headers)

## リクエストを定義

### エンドポイントとヘッダーの設定

### ボディデータを設定する関数を定義

In [125]:
# 従来手法の未登録のユーザをリクエストに含める
def getRequestBasicNonRegisterdUser(numUsers):
    data = {
        "beacons": [],
        "roomId": 1
    }
    for i in range(numUsers):
        hexString = ''.join(random.choices('0123456789abcdef', k=32))
        # ボディデータ
        data["beacons"].append({
            "uuid": hexString,
            "rssi": -49
        })
    print(data)
    return data

# 従来手法の登録済みのユーザをリクエストに含める
def getRequestBasicRegisterdUser(numUsers):
    # ボディデータ
    data = {
        "beacons": [],
        "roomId": 1
    }
    for i in range(numUsers):
        hexString = ''.join(random.choices('0123456789abcdef', k=32))
        # ボディデータ
        data["beacons"].append({
            "uuid": hexString,
            "rssi": -49
        })
    print(data)
    return data

## 計測

### 従来手法での計測

In [151]:
FILE_PATH = "out/basic01.csv"
with open(FILE_PATH, 'w') as f:
  writer = csv.writer(f)
  writer.writerow(['users', 'time', 'user_state', 'status'])

print("start bench")

for i in range(4):
  print(str(i+1) + "/100回目")
  # 100人ランダムな従来ユーザを登録する（PythonでWebAPIを用いて）
  createBasicBeaconUsers(2, 100*i)

  # ランダムなユーザ(登録済み)を10人POSTで投げる
  # start = time.perf_counter()
  # response = requests.post(url, json=getRequestBasicNonRegisterdUser(10), headers=headers)
  # end = time.perf_counter()

  # ランダムなユーザ(未登録)を10人POSTで投げる
  start = time.perf_counter()
  response = requests.post(url, json=getRequestBasicNonRegisterdUser(10), headers=headers)
  end = time.perf_counter()
  with open(FILE_PATH, 'a', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([100*(i+1), end - start, 'NonRegisterd', response.json()])  # 1行追加


# start = time.perf_counter()

# response = requests.post(url, json=getRequestBasicNonRegisterdUser(10), headers=headers)

# end = time.perf_counter()
# print(response.json())
# print("end bench")
# print("実行時間: " + str(end - start) + " sec")

start bench
1/100回目
{'beacons': [{'uuid': 'dda5609ab839a5f1c7c41d3824828bae', 'rssi': -49}, {'uuid': '35703b3aa8f29961124c9f9c7a6d6dc4', 'rssi': -49}, {'uuid': '6eef7f772b16b9041375128ffcd42824', 'rssi': -49}, {'uuid': '968dd25287318038dbf94bedaffd81c5', 'rssi': -49}, {'uuid': 'acebc40a234e91c53336f591719a74af', 'rssi': -49}, {'uuid': '3fbb021121e4b751c5e1592096d23ffd', 'rssi': -49}, {'uuid': 'fd2f0c357dc8314286530c8ef271e67d', 'rssi': -49}, {'uuid': 'b7ac5dc2ccf88ef663f2230e2c718a32', 'rssi': -49}, {'uuid': '68a0a974f4c5c77cfdd0076f90e92cd2', 'rssi': -49}, {'uuid': 'cd77b1d53eb010996db8323ac135cd0e', 'rssi': -49}], 'roomId': 1}
2/100回目
{'beacons': [{'uuid': '80f5281563811936de3bd0f50adb0fb4', 'rssi': -49}, {'uuid': '4cdfe42b5a7a58d6e26a457491ebaba6', 'rssi': -49}, {'uuid': '34976a1d6bf9cd0ace670baf107a09b6', 'rssi': -49}, {'uuid': '9995354ee8bfb571b2c6d8af49721fdb', 'rssi': -49}, {'uuid': 'e83f0a0cc0248c227ea72f9e473e623c', 'rssi': -49}, {'uuid': '717ac4470217345841df677aa00996b4', 'r

### 滞在情報をリセット

In [96]:
# ボディデータ
data = {
    "beacons": [
    ],
    "roomId": 1
}
response = requests.post(url, json=data, headers=headers)